In [1]:
import numpy as np
import pandas as pd
from ztf_data import make_ztf_easy_batch, load_ztf_easy_batch

In [3]:
ztf_batch = load_ztf_easy_batch(thresh_deg=1.0)

In [ ]:
ztf_batch

In [ ]:
ztf_batch.to_hdf('../data/ztf/ztf-easy-batch-n=64-thresh=60arcmin.h5', key='ztf_batch', mode='w')